In [113]:
# importar librerias de manipulacion de datos
import numpy as np 
import pandas as pd
from dateutil.parser import parse

# importar librerias del sistema
import os
from pathlib import Path

root_dir = Path(".").resolve().parent

filename = "llamadas123_julio_2022.csv"
data_dir = "raw"

file_path = os.path.join(root_dir, "data", data_dir, filename)

data = pd.read_csv(file_path, encoding= "latin-1", sep = ";")

data.head(3)

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,RECEPCION
0,CRU-00278353-22,2022-07-01 00:08:59,11,Suba,88,A¤os,FEMENINO,Norte,Accidente Cerebro Vascular,ALTA,2022-07-01 03:40:13
1,CRU-00278360-22,2022-07-01 00:14:58,11,Suba,16,A¤os,MASCULINO,Norte,Herido,ALTA,2022-07-01 03:38:13
2,CRU-00278365-22,2022-07-01 00:21:59,18,Rafael Uribe Uribe,52,A¤os,FEMENINO,Centro Oriente,Evento Respiratorio,CRITICA,2022-07-01 04:08:24


# LIMPIENZA DE DUPLICADOS

In [114]:
#Primero hacemos la limpieza de los duplicados

print("Forma inicial", data.shape)

data = data.drop_duplicates() #Elimina los registros duplicados, dejando solo el primer registro

print("Forma final", data.shape)

Forma inicial (13055, 11)
Forma final (13026, 11)


# CONOCER ESTRUCTURA DE LA TABLA Y DATOS

In [115]:
# Esquema de la tabla para conocer tipos de datos y cantidad de nulos

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13026 entries, 0 to 13054
Data columns (total 11 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   NUMERO_INCIDENTE                   13026 non-null  object
 1   FECHA_INICIO_DESPLAZAMIENTO_MOVIL  13026 non-null  object
 2   CODIGO_LOCALIDAD                   13026 non-null  int64 
 3   LOCALIDAD                          13026 non-null  object
 4   EDAD                               13026 non-null  object
 5   UNIDAD                             13018 non-null  object
 6   GENERO                             13026 non-null  object
 7   RED                                13026 non-null  object
 8   TIPO_INCIDENTE                     13026 non-null  object
 9   PRIORIDAD                          13026 non-null  object
 10  RECEPCION                          13026 non-null  object
dtypes: int64(1), object(10)
memory usage: 1.2+ MB


# LIMPIEZA DE LA COLUMNA UNIDAD

In [116]:
#Vamos a conocer cuantos nulos tenemos y los reemplazamos por un valor, en este caso "SIN_DATO"

data["UNIDAD"].value_counts(dropna=False) #Realiza el conteo sin eliminar los nulos


A¤os        6781
SIN_DATO    6187
Meses         39
NaN            8
Horas          6
Dias           5
Name: UNIDAD, dtype: int64

In [117]:
data["UNIDAD"].fillna("SIN_DATO").value_counts(dropna=False)

A¤os        6781
SIN_DATO    6195
Meses         39
Horas          6
Dias           5
Name: UNIDAD, dtype: int64

In [118]:
# Sobreeescribo la columna UNIDAD que tenia valores nulos por una columna sin nulos
data["UNIDAD"] = data["UNIDAD"].fillna("SIN_DATO")

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13026 entries, 0 to 13054
Data columns (total 11 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   NUMERO_INCIDENTE                   13026 non-null  object
 1   FECHA_INICIO_DESPLAZAMIENTO_MOVIL  13026 non-null  object
 2   CODIGO_LOCALIDAD                   13026 non-null  int64 
 3   LOCALIDAD                          13026 non-null  object
 4   EDAD                               13026 non-null  object
 5   UNIDAD                             13026 non-null  object
 6   GENERO                             13026 non-null  object
 7   RED                                13026 non-null  object
 8   TIPO_INCIDENTE                     13026 non-null  object
 9   PRIORIDAD                          13026 non-null  object
 10  RECEPCION                          13026 non-null  object
dtypes: int64(1), object(10)
memory usage: 1.2+ MB


# LIMPIEAZA DE LA COLUMNA FECHA_INICIO_DESPLAZAMIENTO_MOVIL

In [119]:
# Convertir columna en formato fecha

col = "FECHA_INICIO_DESPLAZAMIENTO_MOVIL"

data[col] = pd.to_datetime(data[col], errors = "coerce")

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13026 entries, 0 to 13054
Data columns (total 11 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   NUMERO_INCIDENTE                   13026 non-null  object        
 1   FECHA_INICIO_DESPLAZAMIENTO_MOVIL  13025 non-null  datetime64[ns]
 2   CODIGO_LOCALIDAD                   13026 non-null  int64         
 3   LOCALIDAD                          13026 non-null  object        
 4   EDAD                               13026 non-null  object        
 5   UNIDAD                             13026 non-null  object        
 6   GENERO                             13026 non-null  object        
 7   RED                                13026 non-null  object        
 8   TIPO_INCIDENTE                     13026 non-null  object        
 9   PRIORIDAD                          13026 non-null  object        
 10  RECEPCION                         

# LIMPIEZA DE LA COLUMNA RECEPCION

In [120]:
#Reiniciamos el indiced e nuestra tabla y creo una funcion que me permitira convertir los formatos a fechas
data = data.reset_index()

def convertir_formato_fecha(str_fecha):
    val_datetime = parse(str_fecha, dayfirst = True)
    return val_datetime

In [121]:
# Ejecuto un ciclo que me permita hacer el cambio de fechas y crearlo en una nueva lista

list_fechas = list()
n_filas = data.shape[0]
for i in range(0, n_filas):
    
    str_fecha = data["RECEPCION"][i]

    try:
        val_datetime = convertir_formato_fecha(str_fecha= str_fecha)
        list_fechas.append(val_datetime)
    except Exception as e:
        list_fechas.append(str_fecha)
        continue

data["RECEPCION_CORREGIDA"] = list_fechas 

data.head(5)

,index,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,RECEPCION,RECEPCION_CORREGIDA
0,0,CRU-00278353-22,2022-07-01 00:08:59,11,Suba,88,A¤os,FEMENINO,Norte,Accidente Cerebro Vascular,ALTA,2022-07-01 03:40:13,2022-01-07 03:40:13
1,1,CRU-00278360-22,2022-07-01 00:14:58,11,Suba,16,A¤os,MASCULINO,Norte,Herido,ALTA,2022-07-01 03:38:13,2022-01-07 03:38:13
2,2,CRU-00278365-22,2022-07-01 00:21:59,18,Rafael Uribe Uribe,52,A¤os,FEMENINO,Centro Oriente,Evento Respiratorio,CRITICA,2022-07-01 04:08:24,2022-01-07 04:08:24
3,3,CRU-00278370-22,2022-07-01 00:38:38,19,Ciudad Bol¡var,SIN_DATO,SIN_DATO,SIN_DATO,Privada,Herido,ALTA,0000-00-00 00:00:00,0000-00-00 00:00:00
4,4,CRU-00278372-22,2022-07-01 00:53:32,1,Usaqun,33,A¤os,MASCULINO,Norte,Intento de Suicidio,ALTA,2022-07-01 03:25:29,2022-01-07 03:25:29


In [122]:
# Convierto la columna creada a formato fecha y luego reemplazo la columna original
data["RECEPCION_CORREGIDA"] = pd.to_datetime(data["RECEPCION_CORREGIDA"], errors = "coerce")

data["RECEPCION"] = data["RECEPCION_CORREGIDA"]

data.head(5)


,index,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,RECEPCION,RECEPCION_CORREGIDA
0,0,CRU-00278353-22,2022-07-01 00:08:59,11,Suba,88,A¤os,FEMENINO,Norte,Accidente Cerebro Vascular,ALTA,2022-01-07 03:40:13,2022-01-07 03:40:13
1,1,CRU-00278360-22,2022-07-01 00:14:58,11,Suba,16,A¤os,MASCULINO,Norte,Herido,ALTA,2022-01-07 03:38:13,2022-01-07 03:38:13
2,2,CRU-00278365-22,2022-07-01 00:21:59,18,Rafael Uribe Uribe,52,A¤os,FEMENINO,Centro Oriente,Evento Respiratorio,CRITICA,2022-01-07 04:08:24,2022-01-07 04:08:24
3,3,CRU-00278370-22,2022-07-01 00:38:38,19,Ciudad Bol¡var,SIN_DATO,SIN_DATO,SIN_DATO,Privada,Herido,ALTA,NaT,NaT
4,4,CRU-00278372-22,2022-07-01 00:53:32,1,Usaqun,33,A¤os,MASCULINO,Norte,Intento de Suicidio,ALTA,2022-01-07 03:25:29,2022-01-07 03:25:29


In [123]:
#Elimino la columna creada de correcion

data = data.drop(["RECEPCION_CORREGIDA"], axis=1)
data.head(5)


,index,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,RECEPCION
0,0,CRU-00278353-22,2022-07-01 00:08:59,11,Suba,88,A¤os,FEMENINO,Norte,Accidente Cerebro Vascular,ALTA,2022-01-07 03:40:13
1,1,CRU-00278360-22,2022-07-01 00:14:58,11,Suba,16,A¤os,MASCULINO,Norte,Herido,ALTA,2022-01-07 03:38:13
2,2,CRU-00278365-22,2022-07-01 00:21:59,18,Rafael Uribe Uribe,52,A¤os,FEMENINO,Centro Oriente,Evento Respiratorio,CRITICA,2022-01-07 04:08:24
3,3,CRU-00278370-22,2022-07-01 00:38:38,19,Ciudad Bol¡var,SIN_DATO,SIN_DATO,SIN_DATO,Privada,Herido,ALTA,NaT
4,4,CRU-00278372-22,2022-07-01 00:53:32,1,Usaqun,33,A¤os,MASCULINO,Norte,Intento de Suicidio,ALTA,2022-01-07 03:25:29


# LIMPIEZA DE LA COLUMNA EDAD

In [124]:
#Verificamos que datos tiene la columna edad

data["EDAD"].value_counts(dropna = False)

SIN_DATO    6187
23           150
22           140
30           134
27           134
            ... 
7             17
98            16
9             15
8             14
97            12
Name: EDAD, Length: 100, dtype: int64

In [125]:
#Reemplazamos los sin datos por valores nulos, para poder convertir la columna a tipo numerico

data["EDAD"] = data["EDAD"].replace({"SIN_DATO": np.nan})
data["EDAD"].value_counts(dropna = False)

NaN    6187
23      150
22      140
30      134
27      134
       ... 
7        17
98       16
9        15
8        14
97       12
Name: EDAD, Length: 100, dtype: int64

In [126]:
#Convertimos la columna a tipo numerico
data["EDAD"] = pd.to_numeric(data["EDAD"], errors = "coerce")

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13026 entries, 0 to 13025
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   index                              13026 non-null  int64         
 1   NUMERO_INCIDENTE                   13026 non-null  object        
 2   FECHA_INICIO_DESPLAZAMIENTO_MOVIL  13025 non-null  datetime64[ns]
 3   CODIGO_LOCALIDAD                   13026 non-null  int64         
 4   LOCALIDAD                          13026 non-null  object        
 5   EDAD                               6839 non-null   float64       
 6   UNIDAD                             13026 non-null  object        
 7   GENERO                             13026 non-null  object        
 8   RED                                13026 non-null  object        
 9   TIPO_INCIDENTE                     13026 non-null  object        
 10  PRIORIDAD                         

# LIMPIEZA DE DATOS EN COLUMNA LOCALIDAD

In [139]:
#Verificamos los datos provenientes de esta columna
data["LOCALIDAD"].value_counts(dropna = False)

Kennedy               1740
Suba                  1323
Engativa              1304
Bosa                  1065
Ciudad Bolivar        1009
Puente Aranda          742
Fontibon               670
Usaquen                667
Rafael Uribe Uribe     601
Usme                   597
San Cristobal          595
Chapinero              507
Tunjuelito             391
Barrios Unidos         389
Los Martires           379
Teusaquillo            361
Santa Fe               356
Antonio Nariño         244
La Candelaria           85
Sumapaz                  1
Name: LOCALIDAD, dtype: int64

In [138]:
#Reemplazo los valores que tienen caracteres especiales con su respectivo nombre
data['LOCALIDAD']= data['LOCALIDAD'].replace(['Fontib¢n','Engativ ',"Ciudad Bol¡var","Usaqun","San Crist¢bal","Los M rtires", "Antonio Nari¤o"],
                                                 ['Fontibon','Engativa',"Ciudad Bolivar","Usaquen","San Cristobal", "Los Martires","Antonio Nariño"])

data["LOCALIDAD"].value_counts(dropna = False)

Kennedy               1740
Suba                  1323
Engativa              1304
Bosa                  1065
Ciudad Bolivar        1009
Puente Aranda          742
Fontibon               670
Usaquen                667
Rafael Uribe Uribe     601
Usme                   597
San Cristobal          595
Chapinero              507
Tunjuelito             391
Barrios Unidos         389
Los Martires           379
Teusaquillo            361
Santa Fe               356
Antonio Nariño         244
La Candelaria           85
Sumapaz                  1
Name: LOCALIDAD, dtype: int64

In [140]:
# Revisamos los datos de la columna aunidad
data["UNIDAD"].value_counts(dropna = False)

A¤os        6781
SIN_DATO    6195
Meses         39
Horas          6
Dias           5
Name: UNIDAD, dtype: int64

In [141]:
#Reemplazo los valores que tienen caracteres especiales con su respectivo nombre
data['UNIDAD']= data['UNIDAD'].replace(["A¤os"],
                                        ["Años"])

data["UNIDAD"].value_counts(dropna = False)

Años        6781
SIN_DATO    6195
Meses         39
Horas          6
Dias           5
Name: UNIDAD, dtype: int64